<a href="https://colab.research.google.com/github/Esantomi/NLP/blob/master/NLP_Day6_seq2seqTranslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 글자 단위(Character-level)로 구현한 seq2seq 번역기

In [ ]:
!pwd

/content


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [ ]:
import os
import pandas as pd

file_path = './fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
lines.sample(5)

,eng,fra,cc
162262,There are frequently earthquakes in Japan.,Il y a fréquemment des tremblements de terre a...,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
162680,What would you do if you were in my place?,Que feriez-vous à ma place ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
181668,My parents have gone to the airport to see my ...,Mes parents sont partis à l'aéroport pour dire...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
181995,All schoolchildren are half price during Chris...,Tous les écoliers sont à moitié prix pendant l...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
43930,Tom asked a question.,Tom posa une question.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [ ]:
lines = lines[['eng', 'fra']][:50000] #5만개 샘플사용
lines.sample(5)

,eng,fra
4360,Can we start?,Pouvons-nous commencer ?
8595,Tom needs you.,Tom a besoin de toi.
7471,I'm beautiful.,Je suis beau.
30686,That's good enough.,C'est suffisamment bon.
31756,We all felt hungry.,Nous avions tous faim.


In [ ]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'  # start-of-string
eos_token = '\n'  # end-of-string
lines.fra = lines.fra.apply(lambda x: '\t' + x + '\n')

print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 50000


,eng,fra
4386,Come join us.,\tVenez vous joindre à nous !\n
1689,Come alone.,\tVenez seuls !\n
16410,You can come in.,\tTu peux entrer.\n
9046,You can do it.,\tVous pouvez le faire.\n
27837,"Get real, will you?","\tSoyez réaliste, voulez-vous ?\n"


In [ ]:
eng_tokenizer = Tokenizer(char_level=True)
# 글자 단위로 토큰화

eng_tokenizer.fit_on_texts(lines.eng)
# 50000개의 행을 가진 eng의 각 행에 토큰화 수행

input_text = eng_tokenizer.texts_to_sequences(lines.eng)
# 단어를 숫자값 인덱스로 변환하여 저장

input_text[:3]

[[19, 3, 8], [19, 3, 8], [19, 3, 8]]

In [ ]:
fra_tokenizer = Tokenizer(char_level=True)
# 글자 단위로 토큰화

fra_tokenizer.fit_on_texts(lines.fra)
# 50000개의 행을 가진 eng의 각 행에 토큰화 수행

target_text = fra_tokenizer.texts_to_sequences(lines.fra)
# 단어를 숫자값 인덱스로 변환하여 저장

target_text[:3]

[[10, 19, 5, 1, 31, 11],
 [10, 15, 5, 12, 16, 29, 2, 14, 11],
 [10, 26, 9, 8, 28, 2, 1, 31, 11]]

In [ ]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1

print('영어 단어장의 크기 :',eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 52
프랑스어 단어장의 크기 : 73


In [ ]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])

print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 22
프랑스 시퀀스의 최대 길이 74


In [ ]:
# 한번에 출력
print('전체 샘플의 수 :', len(lines))
print('영어 단어장의 크기:', eng_vocab_size)
print('프랑스어 단어장의 크기:', fra_vocab_size)
print('영어 시퀀스의 최대 길이:', max_eng_seq_len)
print('프랑스 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 50000
영어 단어장의 크기: 52
프랑스어 단어장의 크기: 73
영어 시퀀스의 최대 길이: 22
프랑스 시퀀스의 최대 길이 74


In [ ]:
encoder_input = input_text

# 종료 토큰 제거
decoder_input = [[char for char in line if char != fra_tokenizer.word_index[eos_token]] for line in target_text]

# 시작 토큰 제거
decoder_target = [[char for char in line if char != fra_tokenizer.word_index[sos_token]] for line in target_text]

In [ ]:
print(decoder_input[:3])  # <eos>토큰 제거
print(decoder_target[:3])  # <sos>토큰 제거

[[10, 19, 5, 1, 31], [10, 15, 5, 12, 16, 29, 2, 14], [10, 26, 9, 8, 28, 2, 1, 31]]
[[19, 5, 1, 31, 11], [15, 5, 12, 16, 29, 2, 14, 11], [26, 9, 8, 28, 2, 1, 31, 11]]


In [ ]:
encoder_input = pad_sequences(encoder_input, maxlen=max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_fra_seq_len, padding='post')

print('영어 데이터의 크기(shape) :', np.shape(encoder_input))
print('프랑스어 입력데이터의 크기 :', np.shape(decoder_input))
print('프랑스어 출력데이터의 크기 :', np.shape(decoder_target))

영어 데이터의 크기(shape) : (50000, 22)
프랑스어 입력데이터의 크기 : (50000, 74)
프랑스어 출력데이터의 크기 : (50000, 74)


In [ ]:
print(encoder_input[0])

[19  3  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [ ]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

print('영어 데이터의 크기 :', np.shape(encoder_input))
print('프랑스어 입력 데이터의 크기 :', np.shape(decoder_input))
print('프랑스어 출력 데이터의 크기 :', np.shape(decoder_target))  # 샘플의 수 

영어 데이터의 크기 : (50000, 22, 52)
프랑스어 입력 데이터의 크기 : (50000, 74, 73)
프랑스어 출력 데이터의 크기 : (50000, 74, 73)


In [ ]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기 :', np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기 :', np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기 :',np.shape(decoder_target))

영어 학습데이터의 크기 : (50000, 22, 52)
프랑스어 학습 입력데이터의 크기 : (50000, 74, 73)
프랑스어 학습 출력데이터의 크기 : (50000, 74, 73)


## 모델 훈련하기

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [ ]:
# LSTM셀의 마지막 time step의 hidden state와 cell state를 디코더 LSTM의 첫번째 hidden state와 cell state전달해주자

encoder_inputs = Input(shape=(None, eng_vocab_size))
# 입력 텐서를 생성

encoder_lstm = LSTM(units= 256, return_state=True)
# hidden state 256인 LSTM을 생성

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_output은 여기서는 불필요

encoder_states = [state_h, state_c]
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도로 저장

In [ ]:
decoder_inputs = Input(shape=(None, fra_vocab_size))
# 입력 텐서 생성

decoder_lstm = LSTM(units=256, return_sequences= True, return_state=True)
# hidden state size 256 디코더 LSTM 생성

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
# decoder output는 모든 timestep의 hidden state

In [ ]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 52)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 73)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 316416      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [ ]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test), batch_size=128, epochs=30)

Epoch 1/30
368/368 [==============================] - 287s 767ms/step - loss: 0.9032 - val_loss: 0.7956
Epoch 2/30
368/368 [==============================] - 277s 752ms/step - loss: 0.5641 - val_loss: 0.6492
Epoch 3/30
368/368 [==============================] - 282s 767ms/step - loss: 0.4678 - val_loss: 0.5676
Epoch 4/30
368/368 [==============================] - 279s 758ms/step - loss: 0.4090 - val_loss: 0.5101
Epoch 5/30
368/368 [==============================] - 281s 762ms/step - loss: 0.3696 - val_loss: 0.4692
Epoch 6/30
368/368 [==============================] - 277s 752ms/step - loss: 0.3413 - val_loss: 0.4432
Epoch 7/30
368/368 [==============================] - 282s 765ms/step - loss: 0.3200 - val_loss: 0.4279
Epoch 8/30
368/368 [==============================] - 276s 751ms/step - loss: 0.3030 - val_loss: 0.4117
Epoch 9/30
368/368 [==============================] - 270s 733ms/step - loss: 0.2891 - val_loss: 0.4034
Epoch 10/30
368/368 [==============================] - 272s 739m

1. 현재 charlevel로 번역기를 구현했는데 word-level로 번역기 완성하기
2. 1번숙제에 실패하거나 포기하신분들은 seq2seq번역기 flow를 그려서 슬랙에 올리기 (자세히 변수까지)
- 링크 참조 : https://wikidocs.net/86900

### 모델 테스트

- 훈련 시에 학습해야 할 타겟 문장을 디코더 모델의 입력, 출력 시퀀스로 넣어 주고, 디코더 모델이 타겟 문장을 한꺼번에 출력하게 할 수 있습니다. 테스트 단계는 불가능!

- 테스트 단계에서 디코더 동작 순서
  - 인코더에 입력 문장을 넣어 마지막 time step의 hidden, cell state를 얻는다.
  - 토큰인 \t를 디코더에 입력한다.
  - 이전 timestep의 출력층의 예측 결과를 현재 timestep의 입력으로 한다.
  - 3을 반복하다가 토큰인 \n가 예측되면 이를 중단한다.

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)
encoder_model.summary()

In [ ]:
decoder_state_input_h = Input(shape=(256,))
# 이전 timestep의 hidden state를 저장하는 텐서

decoder_state_input_c = Input(shape=(256,))
# 이전 timestep의 cell state를 저장하는 텐서

decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장

# decoder_state_inputs를 현재 time step의 초기상태로 사용
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)

# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states = [state_h, state_c]

In [ ]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model= Model(inputs=[decoder_inputs] + decoder_state_inputs, outputs=[decoder_outputs]+decoder_states)
decoder_model.summary()

In [ ]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, fra_vocab_size))
  target_seq[0, 0, fra2idx['\t']] =1

  stop_condition = False
  decoded_sentence = ""

  # stop condition이 True가 될떄까지 루프 반복
  while not stop_condition:
    # 이전 시점의 상태 state_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq]+ states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = idx2fra[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_fra_seq_len):
      stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, sampled_token_index] =1

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]
  return decoded_sentence

In [ ]:
import numpy as np
for seq_index in [3, 50, 100, 300, 1001]:
  # 입력 문장의 인덱스 (자유롭게 바꿔서 테스트 해보세요!)
  input_seq = encoder_input[seq_index: seq_index +1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장 :', lines.eng[seq_index])
  print('정답 문장 :', lines.fra[seq_index][1:len(lines.fra[seq_index])-1])
  # '\t'와 '\n'을 빼고 출력
  print('번역기가 번역한 문장 :', decoded_sentence[:len(decoded_sentence)-1])
  # '\n'을 빼고 출력